In [3]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import math
import random

Setting environment for Gordon


In [4]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))


In [11]:
# build training data

sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(split, i, j1):
    if split==0:
        dist = np.minimum(train_data_normalized_82[i], train_data_normalized_82[j1:]).sum(axis=1)
    elif split==1:
        dist = np.minimum(train_data_normalized_64[i], train_data_normalized_64[j1:]).sum(axis=1)
    elif split==2:
        dist = np.minimum(train_data_normalized_55[i], train_data_normalized_55[j1:]).sum(axis=1)   
    return dist

exprmt = 10

for e in xrange(exprmt):
    
    t = time.time()

    train_data_82 = []
    train_labels_82 = []
    train_fnames_82 = []
    
    train_data_64 = []
    train_labels_64 = []
    train_fnames_64 = []
    
    train_data_55 = []
    train_labels_55 = []
    train_fnames_55 = []

    test_data = []
    test_labels = []
    test_fnames = []

    n_sample = 500
    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(0.8*len(random_indices))
        random_train_82 = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train_82 = len(random_train_82)
        n_test = len(random_test)
        
        random_train_64 = np.random.choice(random_train_82, int(0.6/0.8*len(random_train_82)), replace=False)
        random_train_55 = np.random.choice(random_train_82, int(0.5/0.8*len(random_train_82)), replace=False)
        n_train_64 = len(random_train_64)
        n_train_55 = len(random_train_55)

        train_hists_82 = np.c_[hists0[random_train_82]]
        train_hists_64 = np.c_[hists0[random_train_64]]
        train_hists_55 = np.c_[hists0[random_train_55]]
        test_hists =  np.c_[hists0[random_test]]

        train_files_82 = files[random_train_82].reshape((n_train_82,))
        train_files_64 = files[random_train_64].reshape((n_train_64,))
        train_files_55 = files[random_train_55].reshape((n_train_55,))
        test_files = files[random_test].reshape((n_test,))

        train_data_82.append(train_hists_82)
        train_labels_82.append(np.ones((n_train_82, )) * labels_index[name])
        train_fnames_82.append(train_files_82)
        train_data_64.append(train_hists_64)
        train_labels_64.append(np.ones((n_train_64, )) * labels_index[name])
        train_fnames_64.append(train_files_64)
        train_data_55.append(train_hists_55)
        train_labels_55.append(np.ones((n_train_55, )) * labels_index[name])
        train_fnames_55.append(train_files_55)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train 82: ', n_train_82, 
        print 'Train 64: ', n_train_64, 
        print 'Train 55: ', n_train_55, 
        print 'Test: ', n_test

    train_data_82 = np.concatenate(train_data_82)
    train_labels_82 = np.concatenate(train_labels_82)
    train_fnames_82 = np.concatenate(train_fnames_82)
    n_train_82 = train_data_82.shape[0]
    train_data_64 = np.concatenate(train_data_64)
    train_labels_64 = np.concatenate(train_labels_64)
    train_fnames_64 = np.concatenate(train_fnames_64)
    n_train_64 = train_data_64.shape[0]
    train_data_55 = np.concatenate(train_data_55)
    train_labels_55 = np.concatenate(train_labels_55)
    train_fnames_55 = np.concatenate(train_fnames_55)
    n_train_55 = train_data_55.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data_82.shape
    print train_labels_82.shape
    print train_fnames_82.shape, train_fnames_82.dtype
    print train_fnames_82[0]
    print train_data_64.shape
    print train_labels_64.shape
    print train_fnames_64.shape, train_fnames_64.dtype
    print train_fnames_64[0]
    print train_data_55.shape
    print train_labels_55.shape
    print train_fnames_55.shape, train_fnames_55.dtype
    print train_fnames_55[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized_82 = train_data_82 / train_data_82.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized_82 = np.nan_to_num(train_data_normalized_82)
    train_data_normalized_64 = train_data_64 / train_data_64.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized_64 = np.nan_to_num(train_data_normalized_64)
    train_data_normalized_55 = train_data_55 / train_data_55.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized_55 = np.nan_to_num(train_data_normalized_55)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle_82 = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(0, i, i) 
                                         for i in range(n_train_82)))
    train_dist_triangle_64 = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(1, i, i) 
                                         for i in range(n_train_64)))
    train_dist_triangle_55 = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(2, i, i) 
                                         for i in range(n_train_55)))


    train_dist_mat_82 = np.empty((n_train_82, n_train_82))
    train_dist_mat_82[np.triu_indices(n_train_82)] = train_dist_triangle_82
    r_82 = np.tril_indices(n_train_82)
    train_dist_mat_82[r_82] = train_dist_mat_82.T[r_82]
    print 'Train 82 kernel shape: ', train_dist_mat_82.shape
    
    train_dist_mat_64 = np.empty((n_train_64, n_train_64))
    train_dist_mat_64[np.triu_indices(n_train_64)] = train_dist_triangle_64
    r_64 = np.tril_indices(n_train_64)
    train_dist_mat_64[r_64] = train_dist_mat_64.T[r_64]
    print 'Train 64 kernel shape: ', train_dist_mat_64.shape
                                            
    train_dist_mat_55 = np.empty((n_train_55, n_train_55))
    train_dist_mat_55[np.triu_indices(n_train_55)] = train_dist_triangle_55
    r_55 = np.tril_indices(n_train_55)
    train_dist_mat_55[r_55] = train_dist_mat_55.T[r_55]
    print 'Train 55 kernel shape: ', train_dist_mat_55.shape
    
    bp.pack_ndarray_file(train_data_normalized_82, sift_dir+'/preprocessed/train_level0_82_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels_82, sift_dir+'/preprocessed/train_level0_82_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames_82, sift_dir+'/preprocessed/train_level0_82_e%d_fnames.bp'%e, chunk_size=255*4000)
    bp.pack_ndarray_file(train_dist_mat_82, sift_dir+'/preprocessed/train_level0_82_e%d_spm_dist_mat.bp'%e)
                                            
    bp.pack_ndarray_file(train_data_normalized_64, sift_dir+'/preprocessed/train_level0_64_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels_64, sift_dir+'/preprocessed/train_level0_64_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames_64, sift_dir+'/preprocessed/train_level0_64_e%d_fnames.bp'%e, chunk_size=255*4000)
    bp.pack_ndarray_file(train_dist_mat_64, sift_dir+'/preprocessed/train_level0_64_e%d_spm_dist_mat.bp'%e)
    
    bp.pack_ndarray_file(train_data_normalized_55, sift_dir+'/preprocessed/train_level0_55_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels_55, sift_dir+'/preprocessed/train_level0_55_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames_55, sift_dir+'/preprocessed/train_level0_55_e%d_fnames.bp'%e, chunk_size=255*4000)
    bp.pack_ndarray_file(train_dist_mat_55, sift_dir+'/preprocessed/train_level0_55_e%d_spm_dist_mat.bp'%e)
                                            
    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level0_e%d_data.bp'%e)
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level0_e%d_labels.bp'%e)
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level0_e%d_fnames.bp'%e, chunk_size=255*4000)
    
    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('5N', 1440) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7n', 3444) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7N', 2579) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('12N', 1230) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Pn', 3042) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('VLL', 1287) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('6N', 154) Train 82:  123 Train 64:  92 Train 55:  76 Test:  31
('Amb', 346) Train 82:  276 Train 64:  206 Train 55:  172 Test:  70
('R', 1082) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Tz', 1387) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('RtTg', 2639) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('LRt', 1050) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('LC', 481) Train 82:  384 Train 64:  287 Train 55:  240 Test:  97
('AP', 483) Train 82:  386 Train 6

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:123: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:125: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:127: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:130: RuntimeWarning: invalid value encountered in divide


Train 82 kernel shape:  (5969, 5969)
Train 64 kernel shape:  (4462, 4462)
Train 55 kernel shape:  (3729, 3729)
('BackG', 5105) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('5N', 1440) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7n', 3444) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7N', 2579) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('12N', 1230) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Pn', 3042) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('VLL', 1287) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('6N', 154) Train 82:  123 Train 64:  92 Train 55:  76 Test:  31
('Amb', 346) Train 82:  276 Train 64:  206 Train 55:  172 Test:  70
('R', 1082) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Tz', 1387) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('RtTg', 2639) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('LRt', 1050) Train 82:  400 Train 64:  299 Train 55:  250 T

Experiment #0 Time: 263.068833 seconds


Train 82 kernel shape:  (5969, 5969)
Train 64 kernel shape:  (4462, 4462)
Train 55 kernel shape:  (3729, 3729)
('BackG', 5105) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('5N', 1440) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7n', 3444) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7N', 2579) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('12N', 1230) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Pn', 3042) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('VLL', 1287) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('6N', 154) Train 82:  123 Train 64:  92 Train 55:  76 Test:  31
('Amb', 346) Train 82:  276 Train 64:  206 Train 55:  172 Test:  70
('R', 1082) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Tz', 1387) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('RtTg', 2639) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('LRt', 1050) Train 82:  400 Train 64:  299 Train 55:  250 T

...

Experiment #7 Time: 29.529136 seconds


Train 82 kernel shape:  (5969, 5969)
Train 64 kernel shape:  (4462, 4462)
Train 55 kernel shape:  (3729, 3729)
('BackG', 5105) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('5N', 1440) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7n', 3444) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('7N', 2579) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('12N', 1230) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Pn', 3042) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('VLL', 1287) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('6N', 154) Train 82:  123 Train 64:  92 Train 55:  76 Test:  31
('Amb', 346) Train 82:  276 Train 64:  206 Train 55:  172 Test:  70
('R', 1082) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('Tz', 1387) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('RtTg', 2639) Train 82:  400 Train 64:  299 Train 55:  250 Test:  100
('LRt', 1050) Train 82:  400 Train 64:  299 Train 55:  250 T

Experiment #8 Time: 31.934953 seconds


Train 82 kernel shape:  (5969, 5969)
Train 64 kernel shape:  (4462, 4462)
Train 55 kernel shape:  (3729, 3729)


Experiment #9 Time: 35.252494 seconds
